# Week 1

Required imports:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import networkx as nx

## Geometric probability – "toy" Monte-Carlo

First, I wanted to discuss this topic with a little bit of fun practice!

Consider the following **problem**: 

Let us randomly generate points from a square of size $2\times2$ (so of area $=4$), centered at the origin of the cartesian plane  – for that, we will randomly generate pairs of numbers, $(x,y)$ – both from an interval $(-1,1)$.

These numbers are generated ("sampled" – is a more proper word) from a **uniform** distribution over that interval $(-1,1)$ – that is, the probability of a number falling into an interval $(x,x+dx)\in(-1,1)$ does not depend on the position $x$ of the interval, only on its length $dx$ – and is, in fact, just proportional to the length:

$$p\left(\textrm{Falling into }(x,x+dx)\right) = dx$$ 

**Question:** what is the probability that our point falls within a circle of radius $1$ centered at the origin?

$$p(\sqrt{x^2+y^2}\leq1)=p(x^2+y^2\leq1)=?$$

**Answer:** Of course, if we sample points uniformly within the square, this probability is just the ratio of areas (measures) of the circle (of radius $R=1$ and the square),

$$p = \frac{\textrm{Area(circle)}}{\textrm{Area(square)}} = \frac{\pi R^2}{4} = \frac{\pi}{4}$$

**Idea:** So if, for some reason, you'd like to **(approximately) calculate $\pi$** on your computer, you can just sample a large number $N$ of such points, and the **fraction** of them for which $x^2+y^2\leq 1$, will be $\pi/4$ – multiply the result by $4$ – and you're done!

Let's try it ourselves:

In [ ]:
n_points = 5000

xy = np.random.uniform(low=-1, high = 1, size = 2*n_points).reshape(2,n_points)

within_circle = (xy[0,:]**2+xy[1,:]**2 <= 1.)
outside_circle = ~within_circle

plt.plot(xy[0,within_circle],xy[1,within_circle],'m.')
plt.plot(xy[0,outside_circle],xy[1,outside_circle],'b.')

print('pi = ',4.*(np.where(within_circle)[0].shape[0])/n_points)

This idea lies in the heard of the **Monte-Carlo methods**, which allowed people to calculate some very complicated things "approximately" with random sampling!

### HW Problem 1: 

Can one somehow, using the same idea, calculate another very important mathematical constant – Euler's number, $e=2.718...$?

_If you're not so good at coding (which I doubt) – you can just sketch a theoretical solution :)_

## Random graphs – Erdős–Rényi model, degree distribution

The whole discipline of random graphs started with the work of Erdős and Rényi, when they've introduced the very basic model of a random graph, that is since named after them.

One starts with a fixed **set of vertices**, ${v_1,v_2,...,v_N}$, and also fixes $p$ – the **probability** that a pair of vertices is connected with an **edge**.

Erdős and Rényi were interested (and were the first to explore it) in how the properties of such graphs depend on this fixed edge formation probability, $p$.

Overall, there are $C_N^2 = \frac{N(N-1)}{2}$ possible edges in our graph (we do not consider loops and multi-edges) – for each of them independently, we toss a coin, and with probability $p$ we draw an edge, with probability $1-p$ we don't.

So to sample one random graph, one can simply sample its **adjacency matrix** 

$$ A_{ij} = \begin{cases}
1,\textrm{ if vertices $v_i$ and $v_j$ are connected with an edge},\\
0,\textrm{ if they are not connected with an edge},
\end{cases} $$

Since this matrix is **symmetric** ($A_{ij}=A_{ji}$), one only has to sample its **upper-triangular** part – and get the rest with a **transpose**.

Let us sample several ER-graphs with different values of $p$:

In [ ]:
ps = [0.05,0.1,0.5] 
N = 20 # Number of vertices

for p in ps:
    A = np.random.choice([0,1], N**2, p=[1-p, p]).reshape(N,N)
    A = np.triu(A,k=1) # take the upper-triangular part
    A = A + A.T # add a transpose to itself
    G = nx.from_numpy_matrix(A)
    print('p = ', p,':')
    nx.draw_circular(G)
    print() # empty space for better view
    plt.show()

One thing that is interesting about random graphs is their **degree distribution** – the probability distribution

$$ p(d) = \textrm{probability that a vertex has degree $d$} $$

**Question:** Can we figure out the degree distribution for the ER model?

**Answer:** Well, yes, this is simple. Every vertex has $N-1$ potential neighbours – so $d$ could take values:

$$0, 1, 2, ..., N-1$$

All the potential edges a vertex has are **independent** of each other – so the degree is a **sum** of independent Bernoulli events:

$$d = \sum_{i=1}^{N-1} e_i, ~~~~~ e_i = 
\begin{cases}
1,\textrm{ with probability $p$},\\
0,\textrm{ with probability $1-p$},
\end{cases}
$$

This is what's called a **Bernoulli scheme**.

The degree of a vertex in the ER model has **binomial distribution:**

$$p(d = k) = C_{N-1}^k \, p^k\, (1-p)^{(N-1)-k} $$

For a given adjacency matrix, the degree of a vertex is just the sum of the corresponding row (or column):

$$ d_i = \sum_{i} A_{ij} $$

So let's have a look at how well our theoretical prediction matches with practice – we'll sample a large random graph and look at the **histogram of its degrees**, and compare it to the plot of the binomial distribution:

In [ ]:
from scipy.special import binom

In [ ]:
N = 100
p = 0.2

A = np.random.choice([0,1], N**2, p=[1-p, p]).reshape(N,N)
A = np.triu(A,k=1) # take the upper-triangular part
A = A + A.T # add a transpose to itself

plt.hist(A.sum(axis=0),bins='auto',alpha = 0.4,density=True,color='red',label='experiment')

supp = list(range(N))
bi = [binom(N-1,k)*(p**k)*((1-p)**(N-1-k)) for k in supp]
plt.bar(supp,bi,alpha=0.4,color='blue',label='theory')
plt.legend()

### HW Problem 2*: 
_**(Star * means it is not necessary, but very instructive!)**_

After Erdős and Rényi, many different random graph models were invented. One very famous of them is the **Barabási–Albert** (BA) model.

In it, the vertex set is not fixed initially – rather, this random graph "grows" from two vertices being connected with an edge.

Each time a new vertex $v_{new}$ appears, it is connected (with an edge) to another vertex, $v_j$, with a **probability depending on the degree of $v_j$** – in the simplest case, just **proportional** to it:

$$p(\textrm{edge from $v_{new}$ to $v_j$}) \propto d(v_j)$$

So, for each new vertex $v_{new}$, one has to check the degrees of all the vertices already present – ${d_1,d_2,...,d_N}$ – and connect it to each $v_j$ with a probability

$$p = \frac{d_j}{\sum_{i=1}^N d_j} $$

**Task:** Sample such a graph. 

_(If you don't want to code it yourself, you can use `networkx.barabasi_albert_graph`)_

Plot the degree distribution of it (in **log-log** scale)!